In [3]:
import datetime
import time
from snappy import ProductIO,HashMap,GPF
import os,gc
import snappy
from sentinelsat import SentinelAPI, geojson_to_wkt, read_geojson

In [4]:
import glob
import subprocess
import jpy
from zipfile import ZipFile
from IPython.display import display
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
%matplotlib qt

In [6]:
def get_subset_product(s1_read,x,y,width,height):
    parameters = snappy.HashMap()
    parameters.put('copyMetaData', True)
    parameters.put('region','%s,%s,%s,%s'%(x,y,width,height))
    subset = snappy.GPF.createProduct('Subset',parameters,s1_read)
    return subset
    band_sub = project2img(subset)
    
def do_terrain_correction(source, downsample:bool=False):
    print('\tTerrain correction...')
    parameters = HashMap()
    parameters.put('demName', 'GETASSE30')
    parameters.put('sourceBands', 'Intensity_VH,Intensity_VV,Amplitude_VV,Amplitude_VH')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
#     parameters.put('mapProjection', proj)       # comment this line if no need to convert to UTM/WGS84, default is WGS84
    parameters.put('saveProjectedLocalIncidenceAngle', True)
    parameters.put('saveSelectedSourceBand', True)
    while downsample == 1:                      # downsample: 1 -- need downsample to 40m, 0 -- no need to downsample
        parameters.put('pixelSpacingInMeter', 40.0)
        break
    output = GPF.createProduct('Terrain-Correction', parameters, source)
    return output

def convert_pixel2latlong_sentinel1(project,x,y):
    geoPosType = jpy.get_type('org.esa.snap.core.datamodel.GeoPos')
    geocoding = project.getSceneGeoCoding()
    geo_pos = geocoding.getGeoPos(snappy.PixelPos(x,y),geoPosType())
    return geo_pos.lat, geo_pos.lon

def convert_latlong2pixel_sentinel1(project,lat,long):
    geoPosType = jpy.get_type('org.esa.snap.core.datamodel.PixelPos')
    geocoding = project.getSceneGeoCoding()
    pix_pos = geocoding.getPixelPos(snappy.GeoPos(lat,long),geoPosType())
    return pix_pos.getX(),pix_pos.getY()

def project2img(project,band_name:str='Amplitude_VH',output_type:str='gray'):
    if output_type == 'gray':
        band = project.getBand(band_name)
        w = band.getRasterWidth()
        h = band.getRasterHeight()
        band_data = np.zeros(w*h,np.float32)
        band.readPixels(0,0,w,h,band_data)
        band_data.shape = h,w
    elif output_type == 'rgb':
        
        band_names = ['virtual_red', 'virtual_green', 'virtual_blue']
        h,w = project.getSceneRasterHeight(),project.getSceneRasterWidth()
        band_data = np.zeros((h,w,3),dtype=np.float32)
        for i in range(3):
            band1 = project.getBand(band_names[i])
            w = band1.getRasterWidth()
            h = band1.getRasterHeight()
            band_data1 = np.zeros(w*h,np.float32)
            band1.readPixels(0,0,w,h,band_data1)
            band_data1.shape = h,w
            band_data[:,:,i] = band_data1
            
    elif output_type == 'multiple':
        def get_band(proeject,nm_band):
            band1 = project.getBand(nm_band)
            w = band1.getRasterWidth()
            h = band1.getRasterHeight()
            band_data1 = np.zeros(h*w,dtype=np.float32)
            band1.readPixels(0,0,w,h,band_data1)
            band_data1.shape = h,w
            return band_data1,h,w
                
        band_names = ['Intensity_VV','Intensity_VH']        
        for i in range(len(band_names)):
            bandi,h,w = get_band(project,band_names[i])
            if i == 0:
                band_data = np.zeros((h,w,3),dtype=np.float64)           
            band_data[:,:,i] = bandi
        band_data[:,:,-1] = np.abs(band_data[:,:,0]*band_data[:,:,1])   
#         print(np.max(band_data[:,:,-1]),2**32,2**64)
    return band_data

In [7]:
product_path = r'D:\08 aimagin\satellite image project\sar'
input_s1_files = sorted(list(glob.glob(os.path.join(product_path,'*S1*.zip'),recursive=True)))
# input_s1_files = sorted(list(glob.glob(os.path.join(product_path,'*S1*.dim'),recursive=True)))
# input_s1_files = sorted(list(glob.glob(os.path.join(product_path,'*_TC*.dim'),recursive=True)))

In [6]:
input_s1_files

['D:\\08 aimagin\\satellite image project\\sar\\S1A_IW_GRDH_1SDV_20200523T113602_20200523T113627_032692_03C963_BD84.zip',
 'D:\\08 aimagin\\satellite image project\\sar\\S1A_IW_GRDH_1SDV_20200604T113603_20200604T113628_032867_03CE93_508B.zip',
 'D:\\08 aimagin\\satellite image project\\sar\\S1A_IW_GRDH_1SDV_20210319T113605_20210319T113630_037067_045CF2_F741.zip',
 'D:\\08 aimagin\\satellite image project\\sar\\S1A_IW_GRDH_1SDV_20210518T113608_20210518T113633_037942_047A5D_2ECE.zip',
 'D:\\08 aimagin\\satellite image project\\sar\\S1A_IW_GRDH_1SDV_20210705T113611_20210705T113636_038642_048F4E_C9F7.zip']

In [7]:
input_s1_files = [input_s1_files[-1]]
input_s1_files

['D:\\08 aimagin\\satellite image project\\sar\\S1A_IW_GRDH_1SDV_20210705T113611_20210705T113636_038642_048F4E_C9F7.zip']

In [8]:
name, sensing_mode, product_type, polarization, height, width, band_names = ([] for i in range(7))

In [9]:
for i in input_s1_files:
    if 'GRDH' not in i:
        continue
    print(i)
    sensing_mode.append(i.split('_')[3])
    product_type.append(i.split('_')[4])
    polarization.append(i.split('_')[-6])
    s1_read = snappy.ProductIO.readProduct(i)
    name.append(s1_read.getName())
    height.append(s1_read.getSceneRasterHeight())
    width.append(s1_read.getSceneRasterWidth())
    band_names.append(s1_read.getBandNames())
    
df_s1_read = pd.DataFrame({'Name':name,
                          'Sensing mode':sensing_mode,
                          'Product type':product_type,
                          'Polarization':polarization,
                          'Height':height,
                          'Width':width,
                          'Band names':band_names})
display(df_s1_read)

D:\08 aimagin\satellite image project\sar\S1A_IW_GRDH_1SDV_20210705T113611_20210705T113636_038642_048F4E_C9F7.zip


,Name,Sensing mode,Product type,Polarization,Height,Width,Band names
0,S1A_IW_GRDH_1SDV_20210705T113611_20210705T1136...,1SDV,20210705T113611,1SDV,16793,25399,"[Amplitude_VH, Intensity_VH, Amplitude_VV, Int..."


In [105]:
x, y, width, height = 9500,100,5000,3000
# x, y, width, height = 12000,16000,8000,4000 # coorected terrain

subset = get_subset_product(s1_read,x,y,width,height)
subset = do_terrain_correction(subset, downsample=False)
band_sub = project2img(subset)

print(list(subset.getBandNames()))

	Terrain correction...
['Intensity_VH', 'Intensity_VV', 'Amplitude_VV', 'Amplitude_VH', 'projectedLocalIncidenceAngle']


In [22]:
# band = subset.getBand('Intensity_VV')
# band = subset.getBand('Intensity_VV')
# w = band.getRasterWidth()
# h = band.getRasterHeight()
# band_data = np.zeros(w*h,np.float32)
# band.readPixels(0,0,w,h,band_data)
# band_data.shape = h,w
# band_data_int_vv = band_data.copy()

band_int_vv = project2img(subset,band_name='Intensity_VV')
band_int_vh = project2img(subset,band_name='Intensity_VH')
band_amp_vv = project2img(subset,band_name='Amplitude_VV')
band_amp_vh = project2img(subset,band_name='Amplitude_VH')
# band_ratio_int = np.where(band_int_vh==0,0,band_int_vv/band_int_vh)
# band_ratio_amp = np.where(band_amp_vh==0,0,band_amp_vv/band_amp_vh)
# band_ratio_amp = band_amp_vv/band_amp_vh

In [12]:
from scipy.stats import mode

In [13]:
def rescale28bit(img,bounds):
    out = np.zeros_like(img).astype(np.uint8)
    for i in range(img.shape[2]):
        bound = bounds[i]
        band = img[:,:,i]
        bound_min,bound_max = bound
        band = np.clip(band,bound_min,bound_max)
        print(band.min(),band.max())
        band = 255*(band-bound_min)/(bound_max-bound_min)
        
        out[:,:,i]=band
    return out
        

### dealing df_kanum

In [20]:
def find_status_demolition(args):
    status = ['all','post','remaining','none']
    try:
        return status[np.where(args.notnull())[0][0]]
    except:
        return 'no-info'
        

def find_closest_distance(ix,df):
    p1 = df[['pixel_x','pixel_y']].values[ix]
    if (ix !=0) and (ix!=df.shape[0]-1):
        p_all = np.vstack((df[['pixel_x','pixel_y']].values[0:ix],df[['pixel_x','pixel_y']].values[ix+1:]))
        
    elif ix == 0:
        p_all = df[['pixel_x','pixel_y']].values[1:]
    elif ix == df.shape[0]-1:
        p_all = df[['pixel_x','pixel_y']].values[0:-1]
    min_dist = np.sqrt(np.sum(np.abs(p1-p_all)**2,axis=1)).min()
    argmin_dist = np.sqrt(np.sum(np.abs(p1-p_all)**2,axis=1)).argmin()
    if (ix !=0) and (ix!=df.shape[0]-1):
        if ix-argmin_dist>1:
            argmin_dist+=0
        else:
            argmin_dist+=1
    elif ix == 0:
        argmin_dist+=1
    elif ix == df.shape[0]-1:
        argmin_dist = argmin_dist
    return argmin_dist, min_dist

def cal_pixel_under_area(img,w,h,x,y,iden):
#     print(x,y)
    x = int(x)
    y=int(y)
    sub_img = img[y-h:y+h,x-w:x+w]
    return iden,img[y,x],np.mean(sub_img),np.max(sub_img),np.std(sub_img)

class Formatter(object):
    def __init__(self, im, project):
        self.im = im
        self.project = project
    def __call__(self, x, y):
        z = self.im.get_array()[int(y), int(x)]
        lat, lon = convert_pixel2latlong_sentinel1(self.project,x,y)
        return 'x={:.01f}, y={:.01f}, z={:.05f} || lat:{:.06f}, long:{:.06f}'.format(x, y, z,lat,lon)

class FormatterScatter(object):
    def __init__(self, im, df):
        self.im = im
        self.df = df
    def __call__(self, x, y):
        try:
            pix_x = self.df[self.df['max_pix_xy']==x]['pixel_x'].values[0]
            pix_y = self.df[self.df['max_pix_xy']==y]['pixel_y'].values[0]
        except Exception as e:
            pix_x,pix_y = 88888,88888
        return 'max={:.01f}, mean={:.01f} \n pix_x:{:.06f}, pix_y:{:.06f}'.format(x, y, pix_x,pix_y)
    

In [16]:
status = ['all','post','remaining','none']
kanum_path = r'D:\08 aimagin\satellite image project\sat\coordinate_khanum'
df_all = pd.DataFrame()
countIdStart = 0
status = ['all','post','remaining','none']
for i in os.listdir(kanum_path):
    if '_Data2' not in i:
        continue
    df = pd.read_csv(os.path.join(kanum_path,i))
    df['from_file']=i
    df['result_demolition_status'] = df[['result_demolition_all','result_demolition_post', 'result_demolition_remaining','result_demolition_none']].apply(lambda x:find_status_demolition(x),axis=1)
#     df = df[df['result_demolition_none']=='Y']
    countIdStop = countIdStart+df.shape[0]
    rangeId = np.arange(countIdStart,countIdStop,1)
    countIdStart=countIdStop
    df['ID']=rangeId
    df_all = pd.concat([df_all,df])

In [17]:
band_sub_added1 = band_int_vv.copy()
band_sub_added2 = band_int_vh.copy()
# band_sub_added2 = band_amp_vv.copy()#threshed.copy()
# band_sub_added = cv2.cvtColor(band_sub_added,cv2.COLOR_GRAY2RGB)
for i0,df_allx in df_all.groupby('from_file'):
#     if i0 != 'Tachang_Data2.csv':
#         continue
    print(i0)
    for i in range(df_allx.shape[0]):
        if df_allx['result_demolition_status'].iloc[i] != 'none':
            continue
        lat,long = df_allx['latitude'].iloc[i],df_allx['longitude'].iloc[i]
        pixel_pos2 = convert_latlong2pixel_sentinel1(subset,lat,long)
    #     print(lat,long)
    #     band_sub_added = cv2.rectangle(band_sub_added,(int(pixel_pos2[0]),int(pixel_pos2[1])),(int(pixel_pos2[0]+20),int(pixel_pos2[1]-20)),(255,0,0),1)
    #     band_sub_added = cv2.circle(band_sub_added,(int(pixel_pos2[0]),int(pixel_pos2[1])),10,(255,0,0),2)
        band_sub_added1 = cv2.circle(band_sub_added1,(int(pixel_pos2[0]),int(pixel_pos2[1])),5,200000,1)
        band_sub_added2 = cv2.circle(band_sub_added2,(int(pixel_pos2[0]),int(pixel_pos2[1])),5,100000,1)

#     break

Kanchanadit_Data2.csv
Muang_Data2.csv
PunPin_Data2.csv
Tachang_Data2.csv


In [1182]:
# fig = plt.figure(figsize=(10,30))
fig,ax = plt.subplots(nrows=1, ncols=2,figsize=(10,5))
im1 = ax[0].imshow(band_sub_added1,cmap='gray',vmin=5000,vmax=100000)
im2 = ax[1].imshow(band_sub_added2,cmap='gray',vmin=5000,vmax=100000)
ax[0].format_coord = Formatter(im1,subset)
ax[1].format_coord = Formatter(im2,subset)
plt.show()
# plt.imshow(band_sub_added,cmap='gray',vmin=5000,vmax=10000)

In [18]:
df_all['pixel_coordinate'] = df_all[['latitude','longitude']].apply(lambda x:convert_latlong2pixel_sentinel1(subset,*x),axis=1)
df_all['pixel_x'] = df_all['pixel_coordinate'].apply(lambda x:x[0])
df_all['pixel_y'] = df_all['pixel_coordinate'].apply(lambda x:x[1])

In [19]:
df_all.reset_index(drop=True,inplace=True)
df_all['closest2point_index'] = 'kkk'
df_all['closest2point_pixel'] = 'kkk'
for i in range(df_all.shape[0]):
    argmin_dist, min_dist = find_closest_distance(i,df_all)
    df_all['closest2point_index'].iloc[i]=argmin_dist
    df_all['closest2point_pixel'].iloc[i]=min_dist
#     print(argmin_dist, min_dist)
#     if i == 2:
#         break
    

C:\Users\rooshdee\Anaconda3\envs\aimagin-sat\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [843]:
# df_all[df_all['max_pix_xy'] == df_all['max_pix_xy'].min()][['from_file','latitude','longitude','pixel_x','pixel_y','closest2point_index','closest2point_pixel']]

In [857]:
# df_all[df_all['closest2point_pixel']<4][['from_file','latitude','longitude','pixel_x','pixel_y','closest2point_index','closest2point_pixel','result_demolition_status']]

In [858]:
# plt.figure()
# plt.hist(df_all['closest2point_pixel'],range=(0,20))

In [40]:
def get_stat_kanum_area(df,bands_val,bands_name,w,h,suffix:str='none'):
    if suffix == 'none':
        bands_name = bands_name
        x_col,y_col = 'pixel_x','pixel_y'
    else:
        bands_name = [i+'_'+suffix for i in bands_name]
        x_col,y_col = 'pixel_x_'+suffix,'pixel_y_'+suffix
    for i,j in zip(bands_val,bands_name):
        df['value_at_xy_'+j]=df[[x_col,y_col,'ID']].apply(lambda x: cal_pixel_under_area(i,w,h,*x),axis=1)
        df['val_pix_xy_'+j] = df['value_at_xy_'+j].apply(lambda x:x[1])
        df['mean_pix_xy_'+j] = df['value_at_xy_'+j].apply(lambda x:x[2])
        df['max_pix_xy_'+j] = df['value_at_xy_'+j].apply(lambda x:x[3])
        df['std_pix_xy_'+j] = df['value_at_xy_'+j].apply(lambda x:x[4])
        del df['value_at_xy_'+j]
    return df
    
    

In [73]:
w, h = 5, 5
band_int_vv_norm = np.where(band_int_vv==0,0,(band_int_vv-band_int_vv.min())/(band_int_vv.max()-band_int_vv.min()))
band_int_vh_norm = np.where(band_int_vh==0,0,(band_int_vh-band_int_vh.min())/(band_int_vh.max()-band_int_vh.min()))
band_amp_vv_norm = np.where(band_amp_vv==0,0,(band_amp_vv-band_amp_vv.min())/(band_amp_vv.max()-band_amp_vv.min()))
band_amp_vh_norm = np.where(band_amp_vh==0,0,(band_amp_vh-band_amp_vh.min())/(band_amp_vh.max()-band_amp_vh.min()))

# bands_val = [band_int_vv,band_int_vh,band_amp_vv,band_amp_vh]
bands_val = [band_int_vv_norm,band_int_vh_norm,band_amp_vv_norm,band_amp_vh_norm]
bands_name = ['int_vv','int_vh','amp_vv','amp_vh']
df_all = get_stat_kanum_area(df_all,bands_val,bands_name,w,h,suffix = 'none')
# for i,j in zip(bands_val,bands_name):
#     df_all['value_at_xy_'+j]=df_all[['pixel_x','pixel_y','ID']].apply(lambda x: cal_pixel_under_area(i,w,h,*x),axis=1)
#     df_all['val_pix_xy_'+j] = df_all['value_at_xy_'+j].apply(lambda x:x[1])
#     df_all['mean_pix_xy_'+j] = df_all['value_at_xy_'+j].apply(lambda x:x[2])
#     df_all['max_pix_xy_'+j] = df_all['value_at_xy_'+j].apply(lambda x:x[3])
#     df_all['std_pix_xy_'+j] = df_all['value_at_xy_'+j].apply(lambda x:x[4])
#     del df_all['value_at_xy_'+j]

In [1185]:
%matplotlib inline

In [74]:
fig = plt.figure(figsize=(16,16))
plots = ['val_pix_xy_int_vv', 'mean_pix_xy_int_vv','max_pix_xy_int_vv','std_pix_xy_int_vv', 
         'val_pix_xy_int_vh', 'mean_pix_xy_int_vh','max_pix_xy_int_vh','std_pix_xy_int_vh', 
         'val_pix_xy_amp_vv', 'mean_pix_xy_amp_vv','max_pix_xy_amp_vv','std_pix_xy_amp_vv', 
         'val_pix_xy_amp_vh', 'mean_pix_xy_amp_vh','max_pix_xy_amp_vh','std_pix_xy_amp_vh'
        ]
for i in range(len(plots)):
    fig.add_subplot(4,4,i+1)
    sns.boxplot(x ='result_demolition_status' ,y = plots[i],data = df_all)

In [575]:
fig,ax = plt.subplots()
im1 = ax.scatter(df_all['max_pix_xy'],df_all['mean_pix_xy'])
ax.format_coord = FormatterScatter(im1,df_all)
plt.show()

In [27]:
df_all.to_csv(os.path.join(kanum_path,'processed_kanchanadit-muang-punpin-tachang_รวมทุกdemolition-status.csv'))

In [895]:
%matplotlib qt

In [965]:
plt.figure()
plt.imshow(band_closed,cmap='gray')

In [47]:
def get_mapping_coordinate_label(img,x,y,w,h,diff):
    tmp = img[y-w:y+w,x-w:x+w].copy()   
    x0,y0 = x-w,y-h
    tmp = (((tmp-tmp.min())/(tmp.max()-tmp.min()))*255).astype(np.uint8)
    _, thresh = cv2.threshold(tmp, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    mask = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)[0]
    assert len(mask[0] == 1)
    xi,yi,wi,hi = cv2.boundingRect(mask[0])
    return (x0+xi-diff,y0+yi-diff),(x0+xi+wi+diff,y0+yi+hi+diff)

def find_distance_2pnts(a,b):
    (x1,y1),(x2,y2) = a,b
    return np.sqrt(np.sum([(x1-x2)**2,(y1-y2)**2]))

def check_if_kanumcoord_at_center(img,x_tmp,y_tmp,w,h,thr):
    
    
    tmp = img[y_tmp-w:y_tmp+w,x_tmp-w:x_tmp+w].copy()
    ym,xm = np.where(tmp==tmp.max())
    xm,ym = xm[0],ym[0]
    xc,yc = tmp.shape[0]*0.5,tmp.shape[0]*0.5
    d1 = find_distance_2pnts((xm,ym),(0,0))
    d2 = find_distance_2pnts((xm,ym),(tmp.shape))
    if np.abs(d2-d1)/tmp.shape[0] > thr:
        x_return, y_return = (int(x_tmp-(xc-xm)),int(y_tmp-(yc-ym)))
    else:
        x_return, y_return = (x_tmp,y_tmp)
    return (x_return, y_return)
    
    

In [1276]:
%matplotlib qt

In [65]:
w, h = 5,5
diff = 3
thr = 0.2
img_tmp2 = band_int_vv.copy()
img_tmp3 = band_int_vv.copy()
df_all['pixel_x_new']=999
df_all['pixel_y_new']=999
for i in range(df_all.shape[0]):
#     if df_all['result_demolition_status'].iloc[i] != 'none':
#         continue
    x_tmp0,y_tmp0 = df_all[['pixel_x','pixel_y']].iloc[i].values
    x_tmp0, y_tmp0 = int(x_tmp0), int(y_tmp0)
    x_tmp, y_tmp = check_if_kanumcoord_at_center(band_int_vv,x_tmp0,y_tmp0,w,h,thr)
    a,b = get_mapping_coordinate_label(band_int_vv,x_tmp,y_tmp,w,h,diff)
#     df_all['pixel_coordinate_new'].iloc[i] = (x_tmp,y_tmp)
    df_all['pixel_x_new'].iloc[i] = x_tmp
    df_all['pixel_y_new'].iloc[i] = y_tmp
    
    img_tmp2 = cv2.rectangle(img_tmp2,(x_tmp0-w,y_tmp0-h),(x_tmp0+w,y_tmp0+h),1e5,1)
    img_tmp3 = cv2.rectangle(img_tmp3,a,b,1e5,1)
    
fig,ax = plt.subplots(1,2)
# fig.add_subplot(1,2,1)
im1 = ax[0].imshow(img_tmp2,cmap='gray',vmin=5000,vmax=1e5)
# fig.add_subplot(1,2,2)
im2 = ax[1].imshow(img_tmp3,cmap='gray',vmin=5000,vmax=1e5)
ax[0].format_coord = Formatter(im1,subset)
ax[1].format_coord = Formatter(im2,subset)

C:\Users\rooshdee\Anaconda3\envs\aimagin-sat\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [66]:
df_all = get_stat_kanum_area(df_all,bands_val,bands_name,w,h,suffix = 'new')

In [44]:
fig = plt.figure(figsize=(16,16))
plots = ['val_pix_xy_int_vv_new', 'mean_pix_xy_int_vv_new','max_pix_xy_int_vv_new','std_pix_xy_int_vv_new', 
         'val_pix_xy_int_vh', 'mean_pix_xy_int_vh','max_pix_xy_int_vh_new','std_pix_xy_int_vh_new', 
         'val_pix_xy_amp_vv', 'mean_pix_xy_amp_vv','max_pix_xy_amp_vv_new','std_pix_xy_amp_vv_new', 
         'val_pix_xy_amp_vh_new', 'mean_pix_xy_amp_vh_new','max_pix_xy_amp_vh_new','std_pix_xy_amp_vh_new'
        ]
for i in range(len(plots)):
    fig.add_subplot(4,4,i+1)
    sns.boxplot(x ='result_demolition_status' ,y = plots[i],data = df_all)

In [67]:
df_all['pixel_coordinate_new'] = df_all[['pixel_x_new','pixel_y_new']].apply(lambda x: (x[0],x[1]),axis=1)
df_all['check_difference_new_old_kanum_coordinate'] = \
df_all[['pixel_x','pixel_y','pixel_x_new','pixel_y_new']].apply(lambda x: find_distance_2pnts((x[0],x[1]),(x[2],x[3])),axis=1)

In [71]:
df_all['check_difference_new_old_kanum_coordinate'].mean()

2.77266562517328

In [69]:
df_all[['pixel_x','pixel_y','pixel_x_new','pixel_y_new']].head()

,pixel_x,pixel_y,pixel_x_new,pixel_y_new
0,2963.805600,2618.304326,2961,2622
1,2927.008324,2715.090439,2927,2715
2,3414.031096,2845.890841,3414,2845
3,3394.550185,2833.522009,3397,2828
4,3388.984211,2850.219932,3388,2848


In [84]:
plt.hist(df_all[df_all['result_demolition_status']=='none']['std_pix_xy_int_vh_new'])

(array([194.,  27.,   7.,   2.,   2.,   1.,   2.,   0.,   0.,   2.]),
 array([ 287.53756714,  882.27814636, 1477.01872559, 2071.75930481,
        2666.49988403, 3261.24046326, 3855.98104248, 4450.7216217 ,
        5045.46220093, 5640.20278015, 6234.94335938]),
 <BarContainer object of 10 artists>)

In [88]:
# dfx = df_all.groupby('result_demolition_status').get_group('none')
# dfx[dfx['std_pix_xy_int_vh_new']>500][['from_file','latitude','longitude']]

## เจ้าท่า

In [661]:
fl_excel = 'จำนวนคดี-สภ.ท่าฉาง+การสำรวจ+ขนำ 89 ขนำ'
xl = pd.ExcelFile(os.path.join(kanum_path,fl_excel+'.xlsx'), engine='openpyxl')

In [715]:
df_jaota = pd.DataFrame()
count=0
for i in xl.sheet_names:
    tmp = xl.parse(i,header=1)
    tmp['from_file'] = i
    tmp = tmp[tmp['พิกัด (ทศนิยม)'].notnull()]
    count+=tmp.shape[0]
    df_jaota = pd.concat([df_jaota,tmp])
df_jaota.to_csv(os.path.join(kanum_path,'จำนวนคดี-สภ.ท่าฉาง+การสำรวจ+ขนำ 89 ขนำ.csv'))

In [740]:
df_jaota['latitude'] = df_jaota['พิกัด (ทศนิยม)'].apply(lambda x: float(x.split(', ')[0]))
df_jaota['longitude'] = df_jaota['พิกัด (ทศนิยม)'].apply(lambda x: float(x.split(', ')[1]))
df_jaota['pixel_xy'] = df_jaota[['latitude','longitude']].apply(lambda x: convert_latlong2pixel_sentinel1(subset,*x),axis=1)
df_jaota['pixel_x'] = df_jaota['pixel_xy'].apply(lambda x: x[0])
df_jaota['pixel_y'] = df_jaota['pixel_xy'].apply(lambda x: x[1])

In [743]:
def find_kanum_distance_sornchon_and_jaota(p1,dfx):
    p_all = dfx[['pixel_x','pixel_y']].values
    min_dist = np.sqrt(np.sum(np.abs(p1-p_all)**2,axis=1)).min()
    argmin_dist = np.sqrt(np.sum(np.abs(p1-p_all)**2,axis=1)).argmin()
    return argmin_dist,min_dist

In [1113]:
df_all['closestwithjaota_index'] = 999
df_all['closestwithjaota_pixel'] = 999
df_all['pixel_x_jaota'] = 999
df_all['pixel_y_jaota'] = 999
df_all['latitude_jaota'] = 999
df_all['longitude_jaota'] = 999

for i in range(df_all.shape[0]):
    xi = df_all[['pixel_x','pixel_y']].values[i]
    argmin_dist,min_dist = find_kanum_distance_sornchon_and_jaota(xi,df_jaota)
    df_all['closestwithjaota_index'].iloc[i] = argmin_dist
    df_all['closestwithjaota_pixel'].iloc[i] = min_dist
    df_all['pixel_x_jaota'].iloc[i] = df_jaota['pixel_x'].iloc[argmin_dist]
    df_all['pixel_y_jaota'].iloc[i] = df_jaota['pixel_y'].iloc[argmin_dist]
    df_all['latitude_jaota'].iloc[i] = df_jaota['latitude'].iloc[argmin_dist]
    df_all['longitude_jaota'].iloc[i] = df_jaota['longitude'].iloc[argmin_dist]
    
#     break

C:\Users\rooshdee\Anaconda3\envs\aimagin-sat\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [773]:
# df_jaota.head()

In [89]:
# df_all[df_all['closest2point_pixel']<4].sort_values(by = ['closestwithjaota_pixel'])[['from_file','pixel_x','pixel_y','closest2point_index','closest2point_pixel','pixel_x_jaota','pixel_y_jaota',
#                              'result_demolition_status','latitude_jaota','longitude_jaota', 'closestwithjaota_pixel','closestwithjaota_index']]

In [1120]:
# df_all[df_all['from_file']=='Tachang_Data2.csv']

# water masking

In [8]:
def do_apply_orbit_file(source):
    print('\tApply orbit file...')
    parameters = HashMap()
    parameters.put('Apply-Orbit-File', True)
    output = GPF.createProduct('Apply-Orbit-File', parameters, source)
    return output
def do_thermal_noise_removal(source):
    print('\tThermal noise removal...')
    parameters = HashMap()
    parameters.put('removeThermalNoise', True)
    output = GPF.createProduct('ThermalNoiseRemoval', parameters, source)
    return output
def do_calibration(source, polarization, pols):
    print('\tCalibration...')
    parameters = HashMap()
    parameters.put('outputSigmaBand', True)
    if polarization == 'DH':
        parameters.put('sourceBands', 'Intensity_HH,Intensity_HV')
    elif polarization == 'DV':
        parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
    elif polarization == 'SH' or polarization == 'HH':
        parameters.put('sourceBands', 'Intensity_HH')
    elif polarization == 'SV':
        parameters.put('sourceBands', 'Intensity_VV')
    else:
        print("different polarization!")
    parameters.put('selectedPolarisations', pols)
    parameters.put('outputImageScaleInDb', False)
    output = GPF.createProduct("Calibration", parameters, source)
    return output
def do_speckle_filtering(source):
    print('\tSpeckle filtering...')
    parameters = HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 3)
    parameters.put('filterSizeY', 3)
    output = GPF.createProduct('Speckle-Filter', parameters, source)
    return output
def do_terrain_correction(source,band_nm, downsample:bool=False):
    print('\tTerrain correction...')
    parameters = HashMap()
    parameters.put('demName', 'GETASSE30')
    parameters.put('sourceBands', band_nm)
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
#     parameters.put('mapProjection', proj)       # comment this line if no need to convert to UTM/WGS84, default is WGS84
    parameters.put('saveProjectedLocalIncidenceAngle', True)
    parameters.put('saveSelectedSourceBand', True)
    while downsample == 1:                      # downsample: 1 -- need downsample to 40m, 0 -- no need to downsample
        parameters.put('pixelSpacingInMeter', 40.0)
        break
    output = GPF.createProduct('Terrain-Correction', parameters, source)
    return output

def get_band_names(project):
    band_nm = ''
    for i in list(project.getBandNames()):
        if i != len(list(project.getBandNames())):
            band_nm += i+','
        else:
            band_nm += i
    return band_nm

In [9]:
product_path = r'D:\08 aimagin\satellite image project\sar'
input_s1_files = sorted(list(glob.glob(os.path.join(product_path,'*GRD*.zip'),recursive=True)))
input_s1_files = [input_s1_files[-1]]

In [10]:
name, sensing_mode, product_type, polarization, height, width, band_names = ([] for i in range(7))
for i in input_s1_files:
    if 'GRDH' not in i:
        continue
    print(i)
    sensing_mode.append(i.split('_')[3])
    product_type.append(i.split('_')[4])
    polarization.append(i.split('_')[-6])
    s1_read = snappy.ProductIO.readProduct(i)
    name.append(s1_read.getName())
    height.append(s1_read.getSceneRasterHeight())
    width.append(s1_read.getSceneRasterWidth())
    band_names.append(s1_read.getBandNames())
    if polarization[0][2:4]== 'DV':
        pols = 'VH,VV'
    
df_s1_read = pd.DataFrame({'Name':name,
                          'Sensing mode':sensing_mode,
                          'Product type':product_type,
                          'Polarization':polarization,
                          'Height':height,
                          'Width':width,
                          'Band names':band_names})
display(df_s1_read)

D:\08 aimagin\satellite image project\sar\S1A_IW_GRDH_1SDV_20210705T113611_20210705T113636_038642_048F4E_C9F7.zip


,Name,Sensing mode,Product type,Polarization,Height,Width,Band names
0,S1A_IW_GRDH_1SDV_20210705T113611_20210705T1136...,1SDV,20210705T113611,1SDV,16793,25399,"[Amplitude_VH, Intensity_VH, Amplitude_VV, Int..."


In [11]:
orbit = do_apply_orbit_file(s1_read)
thermal = do_thermal_noise_removal(orbit)
calibrate = do_calibration(thermal, polarization, pols)
filtered = do_speckle_filtering(calibrate)

x, y, width, height = 9500,100,5000,3000
subset2 = get_subset_product(filtered,x,y,width,height)
band_nm = get_band_names(subset2)
subset2 = do_terrain_correction(subset2,band_nm, downsample=False)

	Apply orbit file...
	Thermal noise removal...
	Calibration...
different polarization!
	Speckle filtering...
	Terrain correction...


In [12]:
band_sigma_vv = project2img(subset2,'Sigma0_VV')

In [13]:
plt.figure()
plt.imshow(band_sigma_vv,cmap='gray',vmin=1e-4,vmax=0.7)

In [75]:
# mask = band_sigma_vv.copy()
# plt.hist(mask.flatten(),range=(1e-4,0.7),bins=20)

In [14]:
mask = band_sigma_vv.copy()

thr_max = 0.07
thr_min = 1e-4
mask[mask==0]=thr_max
mask[mask>=thr_max] = thr_max
mask[mask<=thr_min] = thr_min
# mask[mask==thr_max] = thr_min

plt.figure()
plt.imshow(mask,cmap='gray',vmin=thr_min,vmax=thr_max)
# plt.hist(mask[mask!=0].flatten(),range=(5,2e4))

In [15]:
from sklearn.cluster import KMeans

In [16]:
# mask[mask==0]=1e5
kmeans = KMeans(n_clusters=2, random_state=0).fit(mask.reshape(-1,1))

In [17]:
water_mask = kmeans.labels_.reshape(mask.shape)

In [18]:
plt.figure()
plt.imshow(water_mask,cmap='gray')

In [21]:
product_path

'D:\\08 aimagin\\satellite image project\\sar'

In [23]:
water_mask[water_mask==1]=255
cv2.imwrite(os.path.join(product_path,'mask_unclear.png'),water_mask)

True

In [86]:
water_mask[water_mask==0]=5
water_mask[water_mask==1]=0
water_mask[water_mask==5]=1

In [87]:
h,w = water_mask.shape

In [88]:
mask1 = water_mask.copy()
mask1[mask1==1]=255
_,a,b,_ = cv2.floodFill(mask1, None, (0,0), 128);

In [90]:
plt.figure()
plt.imshow(mask1,cmap='gray')

In [ ]:

# cnts = cv2.findContours(mask2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [92]:
def filling_ground(msk,idx1,idx2,diff,val):
    for i,j in zip(idx1,idx2):
        if (msk[i,j-diff] == val) or ((msk[i,j+diff] == val)) or (msk[i-diff,j] == val) or (msk[i+diff,j] == val):
            cv2.floodFill(msk, None, (j,i), val)
            idx1,idx2 = np.where(msk==0)
            repeat=True
            break
        else:
            repeat=False
    return msk, idx1, idx2, repeat

In [93]:
diff=20
ix1,ix2 = np.where(mask1 == 0)
repeat =True
while repeat:
    mask1, ix1, ix2, repeat = filling_ground(mask1,ix1,ix2,diff,128)

In [94]:
mask2 = mask1.copy()
mask2[mask2==0]=255
mask2[mask2==128]=0
mask2[mask2==255]=1
mask2.astype(np.uint8);

In [95]:
cnts = cv2.findContours(mask2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
mask2[mask2==1]=255
for cnt in cnts[0]:
    if (cnt[:,0,0]>4700).all() and (cnt[:,0,1]<2185).all() :
#     if (cnts[0][0][:,0,0]>4700).all():
#         print()
        cv2.drawContours(mask2,[cnt],0,255,-1)
#     else:
#         cv2.drawContours(mask2,[cnt],0,128,-1)
        

In [96]:
plt.figure()
plt.imshow(mask2,cmap='gray')

In [97]:
mask3 = mask2[0:2185,4700:].copy()
mask3[mask3==255]=1
mask3 = mask3.astype(np.uint8)
kernel = np.ones((3,3),np.uint8)
mask3 = cv2.dilate(mask3,kernel,iterations=3)

In [98]:
mask3.shape,np.unique(mask3)

((2185, 873), array([0, 1], dtype=uint8))

In [99]:
mask2[mask2==255]=1
mask2[0:2185,4700:] = mask3.copy()

In [100]:
cnts = cv2.findContours(mask2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [101]:
contour_area = [cv2.contourArea(cnt) for cnt in cnts[0]]

In [123]:
plt.figure()
plt.imshow(mask2,cmap='gray')

In [111]:
plt.figure()
plt.imshow(tmp)

In [268]:
def specific_dilate(msk,x1,x2,y1,y2,kernel_size,iterate):
    ## TO DILATE WHITE AREA
    tmp = msk[y1:y2,x1:x2].copy().astype(np.uint8)
    kernel = np.ones((kernel_size,kernel_size),dtype=np.uint8)
    tmp2 = cv2.dilate(tmp.copy(),kernel,iterations=iterate)
    fig = plt.figure()
    fig.add_subplot(1,2,1)
    plt.imshow(tmp,cmap='gray')
    fig.add_subplot(1,2,2)
    plt.imshow(tmp2,cmap='gray')
    return tmp2

def specific_erode(msk,x1,x2,y1,y2,kernel_size,iterate):
    ## TO ERODE WHITE AREA
    tmp = msk[y1:y2,x1:x2].copy().astype(np.uint8)
    kernel = np.ones((kernel_size,kernel_size),dtype=np.uint8)
    tmp2 = cv2.erode(tmp.copy(),kernel,iterations=iterate)
    fig = plt.figure()
    fig.add_subplot(1,2,1)
    plt.imshow(tmp,cmap='gray')
    fig.add_subplot(1,2,2)
    plt.imshow(tmp2,cmap='gray')
    return tmp2

# mask2[0:300,4800:5050] = tmp.copy()

In [133]:
x1,x2,y1,y2 = 4800,5050,0,300
tmp = specific_dilate(mask2,x1,x2,y1,y2,kernel_size=3,iterate=3)
# mask2[y1:y2,x1:x2]=tmp.copy()
x1,x2,y1,y2 = 5150,54000,1975,2100
tmp = specific_dilate(mask2,x1,x2,y1,y2,kernel_size=7,iterate=3)
# mask2[y1:y2,x1:x2]=tmp.copy()

In [189]:
plt.figure()
plt.imshow(mask2,cmap='gray')

In [197]:
cnts[0][10][:,0,:]

array([[ 662, 3939],
       [ 661, 3940],
       [ 660, 3939],
       [ 661, 3938]], dtype=int32)

In [222]:
cnt[:,0,:][0][0]

4888

In [223]:
pixel_in_contour = [mask3[cnt[:,0,:][0][1],cnt[:,0,:][0][0]] for cnt in cnts[0]]

In [239]:
pixel_in_contour[13963]

1

In [240]:
contour_area = [cv2.contourArea(cnt) for cnt in cnts[0]]

In [242]:
# contour_area[13963]

In [235]:
len(np.where(np.asarray(pixel_in_contour)==1)[0])

5032

In [245]:
from datetime import datetime

In [250]:
mask3 = mask2.copy()
# mask3[mask3==1]=255
cnts = cv2.findContours(mask3,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
tm1 = datetime.now()
print(tm1)
for i in range(len(cnts[0])):
    imgx = np.zeros_like(mask3)
    cv2.drawContours(imgx,cnts[0], i, 1, -1);
    ix,iy = np.where(imgx==1)
    if mask3[ix[0],iy[0]] == 0: #ถ้าเป็นดินอยู่แล้วไม่ต้องเติม
        continue
    elif (ix.min()<3000<ix.max()) and (iy.min()<2000<iy.max()):
        continue
    else:
        cv2.drawContours(mask3,cnts[0], i, 0, -1);
tm2 = datetime.now()
print(tm2)
print((tm2-tm1).total_seconds())

        
# mask3[mask3==255]=0
# mask3[mask3==128]=1

2021-07-23 00:35:39.320799
2021-07-23 01:00:21.297503
1481.976704


In [255]:
mask3_tmp = mask3.copy()
mask3_tmp[mask3_tmp==1]=255

In [256]:
cv2.imwrite(os.path.join(product_path,'mask_after_removecity.png'),mask3_tmp)

True

In [257]:
plt.figure()
plt.imshow(mask3,cmap='gray')

In [266]:
tmp = mask3[800:1150,1600:1900].copy().astype(np.uint8)
# plt.figure()
# plt.imshow(tmp)

In [ ]:
specific_dilate

In [277]:
x1,x2,y1,y2 = 1600,1900,800,1050
tmp = specific_erode(mask3,x1,x2,y1,y2,3,4);
mask3[y1:y2,x1:x2]=tmp.copy()

In [282]:
x1,x2,y1,y2 = 1600,1900,1060,1125
tmp = specific_erode(mask3,x1,x2,y1,y2,3,2);
mask3[y1:y2,x1:x2]=tmp.copy()

In [286]:
x1,x2,y1,y2 = 4400,4750,2200,2400
tmp = specific_dilate(mask3,x1,x2,y1,y2,3,3);
mask3[y1:y2,x1:x2]=tmp.copy()

In [292]:
x1,x2,y1,y2 = 5150,5350,2150,2450
tmp = specific_dilate(mask3,x1,x2,y1,y2,3,3);
mask3[y1:y2,x1:x2]=tmp.copy()

In [296]:
x1,x2,y1,y2 = 3800,3840,2980,3020
tmp = specific_dilate(mask3,x1,x2,y1,y2,3,2);
mask3[y1:y2,x1:x2]=tmp.copy()

In [303]:
tmp = mask3.copy()
cv2.floodFill(tmp,None,(3807,3007),1)
cv2.floodFill(tmp,None,(3828,2992),1)
plt.figure()
plt.imshow(tmp,cmap='gray')
mask3 = tmp.copy()

In [312]:
x1,x2,y1,y2 = 3800,4000,2900,3100
tmp = specific_erode(mask3,x1,x2,y1,y2,7,4);
mask3[y1:y2,x1:x2]=tmp.copy()

In [320]:
cv2.floodFill(mask3,None,(3977,2862),0)
cv2.floodFill(mask3,None,(863,1632),1)
cv2.floodFill(mask3,None,(917,1600),1)
cv2.floodFill(mask3,None,(974,1558),1)
cv2.floodFill(mask3,None,(2660,3198),1)
# plt.figure()
# plt.imshow(mask3,cmap='gray')

(2106,
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 None,
 (2619, 3132, 90, 126))

In [328]:
x1,x2,y1,y2 = 1750,2000,2650,2950
tmp = specific_dilate(mask3,x1,x2,y1,y2,5,2);
mask3[y1:y2,x1:x2]=tmp.copy()

In [336]:
cv2.floodFill(mask3,None,(2541,3174),1)
cv2.floodFill(mask3,None,(2557,3174),1)

(22,
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 None,
 (2557, 3172, 4, 7))

In [343]:
x1,x2,y1,y2 = 700,1500,700,1800
tmp = specific_erode(mask3,x1,x2,y1,y2,7,4);
mask3[y1:y2,x1:x2]=tmp.copy()

In [360]:
x1,x2,y1,y2 = 1100,2600,2600,3800
tmp = specific_erode(mask3,x1,x2,y1,y2,7,7);
mask3[y1:y2,x1:x2]=tmp.copy()

In [361]:
plt.figure()
plt.imshow(mask3,cmap='gray')

In [362]:
x1,y1 = 1579,695
x2,y2 = 5371,2158
m = (y2-y1)/(x2-x1)
c = y2-m*x2

In [363]:
xi,yi = 5000,2000
print(m*xi+c)

2014.863660337553


In [364]:
m*xi+c

2014.863660337553

In [365]:
tmp = mask3.copy()
for i in range(tmp.shape[0]):
    for j in range(tmp.shape[1]):
        if i < m*j+c:
            tmp[i,j]=0

In [366]:
plt.figure()
plt.imshow(tmp,cmap='gray')

In [367]:
tmp[tmp==1]=255
fl_name =r'D:\08 aimagin\satellite image project\sar\gulf_mask_SAR.png'
cv2.imwrite(fl_name,tmp)

True

## edit water mask

In [78]:
mask1 = cv2.imread(os.path.join(product_path,'mask_unclear.png'),0)
mask2 = cv2.imread(os.path.join(product_path,'gulf_mask_SAR.png'),0)

In [31]:
fig = plt.figure()
fig.add_subplot(1,2,1)
plt.imshow(mask1,cmap='gray')
fig.add_subplot(1,2,2)
plt.imshow(mask2,cmap='gray')

In [63]:
cv2.floodFill(mask1,None,(3188,2430),255);

In [92]:
plt.figure()
plt.imshow(mask1,cmap='gray')

In [91]:
plt.figure()
plt.imshow(img1,cmap='gray')

In [64]:
kernel_size = 2
kernel = np.ones((kernel_size,kernel_size),dtype=np.uint8)
y1,y2,x1,x2 = 2400,2500,3125,3250
img1 = mask1[y1:y2,x1:x2].copy()
img1 = img1.astype(np.uint8)
fig = plt.figure()
fig.add_subplot(1,2,1)
plt.imshow(img1,cmap='gray')
img1 = cv2.dilate(img1,kernel,iterations=3)
img1[img1==255]=128
fig.add_subplot(1,2,2)
plt.imshow(img1,cmap='gray')

In [65]:
mask1[mask1==1]=255
mask1[y1:y2,x1:x2] = img1
plt.figure()
plt.imshow(mask1,cmap='gray')

In [59]:
ix_gray = np.where(mask1 == 128)

In [84]:
tmp = mask2.copy()
kernel_size = 3
kernel = np.ones((kernel_size,kernel_size),dtype=np.uint8)
tmp = cv2.erode(tmp,kernel,iterations=4)
tmp[ix_gray] = 0
fig = plt.figure()
fig.add_subplot(1,2,1)
plt.imshow(mask2,cmap='gray')
fig.add_subplot(1,2,2)
plt.imshow(tmp,cmap='gray')

In [81]:
cnts = cv2.findContours(tmp,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [98]:
mask2 = tmp.copy()

In [97]:
plt.figure()
plt.imshow(tmp)

In [88]:
mask
tmp2 = tmp*band_sigma_vv

In [96]:
plt.figure()
plt.imshow(tmp2,cmap='gray',vmin=1e-4,vmax=20)

In [95]:
mask1[mask1==1]=255
plt.figure()
plt.imshow(mask1,cmap='gray')

In [101]:
# mask1

In [104]:
tmp = mask1.copy()
cv2.floodFill(tmp,None,(2661,3196),128)
cv2.floodFill(tmp,None,(2603,3183),128)
plt.figure()
plt.imshow(tmp,cmap='gray')

In [105]:
ix = np.where(tmp==128)
mask2[ix]=0

array([  0, 255], dtype=uint8)

In [118]:
cv2.floodFill(mask2,None,(2646,3231),0)
cv2.floodFill(mask2,None,(2638,3237),0)
# mask2[3235:3241,2634:2640]=0
# mask2[3235:3241,2634:2640]=0
mask2[3218:3221,2655:2658]=0
plt.figure()
plt.imshow(mask2,cmap='gray')

In [110]:
plt.figure()
plt.imshow(mask2,cmap='gray')

In [119]:
tmp2 = mask2*band_sigma_vv
plt.figure()
plt.imshow(tmp2,cmap='gray',vmin=1e-4,vmax=20)

In [121]:
cv2.imwrite(os.path.join(product_path,'gulf_mask_SAR2.png'),mask2)

True